In [51]:

A=torch.load('/Users/avi/Desktop/Food_GVAE-master/E2C_spring_mass/latent_2/control_data_2.pt')

In [52]:
import torch
import numpy as np

In [53]:
A=A[:100]

In [54]:
# store 1st element of each list in a new list states 2nd element of each list in a new list A_mat and 3rd element of each list in a new list B_mat
states=[]
A_mat=[]
B_mat=[]
for i in range(len(A)):
    states.append(A[i][0].detach().numpy()[:4])
    A_mat.append(A[i][1].detach().numpy()[:4,:4])
    B_mat.append(A[i][2].detach().numpy()[:4,:1].squeeze())




In [42]:
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.animation as animation 
import numpy as np 
import time
import control

In [56]:
def state_space_model(A, state_t_minus_1, B, control_input_t_minus_1):
    state_estimate_t = (A @ state_t_minus_1) + (B @ control_input_t_minus_1)
    return state_estimate_t

In [63]:
def get_control_waypoints(waypoints,A_matR,B_matR,R,error_th = 1, verbose = False):
    waypoints = [np.array(i) for i in waypoints]
    trajectory = []
    controls = []
    error = []
    waypoints_reached = []
    actual_state_x = np.array([0,0,0,0])
    
    Q = np.array([[1, 0, 0, 0],[0, 1.0, 0, 0], [0, 0, 1.0, 0],[0,0,0,1.0]])    
    for desired_state_xf,A,B in zip(waypoints,A_matR,B_matR):
        state_error_magnitude = 1e10
        waypoints_reached.append(actual_state_x)
        while state_error_magnitude >= error_th:
            if verbose:
                print(f'Current State = {actual_state_x}')
                print(f'Desired State = {desired_state_xf}')
            state_error = actual_state_x - desired_state_xf
            trajectory.append(actual_state_x)
            state_error_magnitude = np.linalg.norm(state_error)    
            error.append(state_error_magnitude)
            if verbose:
                print(f'State Error Magnitude = {state_error_magnitude}')
            print(B.shape)
            B=B.reshape(4,1)
            optimal_control_input = control.dlqr(A, B, Q, R)[0]
            controls.append(optimal_control_input)
            if verbose: 
                print(f'Control Input = {optimal_control_input}')
            actual_state_x = state_space_model(A, actual_state_x, B, 
                                            optimal_control_input)
            if verbose:
                if state_error_magnitude < error_th:
                    print("\nGoal Has Been Reached Successfully!")
    return(trajectory,controls,waypoints_reached,error)

In [64]:
def get_animation(waypoints,trajectory,verbose=False,marker_txt=True):
    plt.style.use('dark_background')
    fig = plt.figure(figsize=(10, 10)) 
    ax = plt.axes(xlim=(0, 110), ylim=(0, 110)) 
    x1 = np.array(waypoints).T[1]
    x0 = np.array(waypoints).T[0]
    x2 = np.array(waypoints).T[2]
    x3 = np.array(waypoints).T[3]
    n = [i for i in range(len(waypoints))]


    plt.subplots()
    plt.plot(range(len(np.array(waypoints).T[0])), np.array(waypoints).T[0], "-r", label="original")
    plt.plot(range(len(np.array(waypoints).T[0])), np.array(trajectory).T[0], "-g", label="tracking")
    plt.plot(range(len(np.array(waypoints).T[0])), np.array(waypoints).T[1], "-r", label="original")
    plt.plot(range(len(np.array(waypoints).T[0])), np.array(trajectory).T[1], "-g", label="tracking")
    plt.plot(range(len(np.array(waypoints).T[0])), np.array(waypoints).T[2], "-r", label="original")
    plt.plot(range(len(np.array(waypoints).T[0])), np.array(trajectory).T[2], "-g", label="tracking")
    plt.plot(range(len(np.array(waypoints).T[0])), np.array(waypoints).T[3], "-r", label="original")
    plt.plot(range(len(np.array(waypoints).T[0])), np.array(trajectory).T[3], "-g", label="tracking")
    plt.axis("equal")
    plt.xlabel("time")
    plt.ylabel("states")
    plt.legend()

    
    def init(): 
        line.set_data([], []) 
        return line, 
    xdata, ydata = [], [] 
    
    def animate(i): 
        x=range(len(trajectory[i][0]))
        y = trajectory[i][0]
        xdata.append(x) 
        ydata.append(y) 
        line.set_data(xdata, ydata)
        #text_ = "       CONTROL\n"+'Velocity: '+ str(np.round(controls[i][0],2))+'\nAngular-velocity: '+str(np.round(controls[i][1],2))
        #label.set_text(text_)
        if verbose:
            print(i,x,y)
        return line, 
    anim = animation.FuncAnimation(fig, animate, init_func=init, 
                                  frames=len(trajectory), interval=1, blit=True,) 
    anim.save('coil.gif',writer='imagemagick')
    plt.subplots()
    plt.plot(np.arange(len(controls)), np.array(controls).T[0], "-r", label="control")
    plt.xlabel("Time [s]")
    plt.ylabel("control")
    plt.show()
    

In [67]:
# store 1st element of each list in a new list states 2nd element of each list in a new list A_mat and 3rd element of each list in a new list B_mat
states=[]
A_mat=[]
B_mat=[]
for i in range(len(A)):
    states.append(A[i][0].detach().numpy()[:4])
    A_mat.append(A[i][1].detach().numpy()[:4,:4])
    B_mat.append(A[i][2].detach().numpy()[:4,:1].squeeze())

R = np.array([[1.0, 0],[0, 1.0]])
trajectory,controls,waypoints_reached,error = get_control_waypoints(states,A_mat,B_mat,R,error_th = 1, verbose=False)
get_animation(waypoints,trajectory)

(4,)


ControlDimension: Incompatible dimensions of R matrix